In [51]:
import os
import gym
import gym_donkeycar
import numpy as np
import random
import tensorflow
import shimmy
from rich.live import Live
import stable_baselines3

In [52]:
DONKEY_GYM = True
DONKEY_SIM_PATH = "remote"
DONKEY_GYM_ENV_NAME = "donkey-generated-track-v0"
body_style = "car01"
body_rgb = (128, 128, 128)
car_name = "PPO"
font_size = 100
WEB_CONTROL_PORT = int(os.getenv("WEB_CONTROL_PORT", 8887))
port = 9091

In [53]:
conf = { "DONKEY_GYM":DONKEY_GYM, "DONKEY_SIM_PATH":DONKEY_SIM_PATH, "DONKEY_GYM_ENV_NAME":DONKEY_GYM_ENV_NAME, 
         "body_style":body_style, "body_rgb":body_rgb, "car_name":car_name, "font_size":font_size, 
         "WEB_CONTROL_PORT" : WEB_CONTROL_PORT, "port" : port }

In [54]:
env = gym.make("donkey-minimonaco-track-v0", conf=conf)

starting DonkeyGym env

Setting default: start_delay 5.0

Setting default: max_cte 8.0

Setting default: frame_skip 1

Setting default: cam_resolution (120, 160, 3)

...

Setting default: host localhost

Setting default: steer_limit 1.0

Setting default: throttle_min 0.0

Setting default: throttle_max 1.0

INFO:gym_donkeycar.core.client:connecting to localhost:9091 
INFO:gym_donkeycar.envs.donkey_sim:on need car config
INFO:gym_donkeycar.envs.donkey_sim:sending car config.
INFO:gym_donkeycar.envs.donkey_sim:sim started!


In [47]:
from stable_baselines3 import PPO
model = PPO("CnnPolicy", env, verbose=1, )

Using cuda device

Wrapping the env with a `Monitor` wrapper

Wrapping the env in a DummyVecEnv.

Wrapping the env in a VecTransposeImage.

In [49]:
model.learn(total_timesteps=20_000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 77.9     |
|    ep_rew_mean     | 49.6     |
| time/              |          |
|    fps             | 15       |
|    iterations      | 1        |
|    time_elapsed    | 135      |
|    total_timesteps | 2048     |
---------------------------------

KeyboardInterrupt: 

In [ ]:
model.save('/')

In [ ]:
evaluate_policy(model , env , n_eval_episodes=20)

In [ ]:
model.set_env(env)

In [6]:
# for i in range(1):
#     # PLAY
#     obs = env.reset()
#     for t in range(100):
#       x1 = random.random()
#       x2 = random.random()
#       action = np.array([x1, x2]) # drive straight with small speed
#       # execute the action
#       obs, reward, done, info = env.step(action)

# # Exit the scene
# env.close()

In [50]:
env.close()